# Missing Migrants Twitter Classification

In this project we are building a classifier to help the [Missing Migrants project](https://missingmigrants.iom.int/). 

IOM needs to find news about migrants that die or go missing, all around the world. One of the sources is Twitter. They have several search terms that they can use ("missing migrant", "migrant death", "migrant drown", etc.), but the terms return many tweets that are not actual _reports_ of new migrants that have gone missing. 

Our job is to build a classifier that will get a set of tweets, and return a classification: 1 if it is a specific news report about a missing or dead migrant, 0 otherwise. 

We have a training set, ~2200 labelled tweets, on which to train the classifier. 

In [ ]:
! pip install langdetect

In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, VectorizerMixin
from utils import ngrammer, clean_html, get_errors

In [ ]:
%reload_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

pd.set_option('max_colwidth', 120)

In [ ]:
from utils import clean_html
from sklearn.feature_extraction.text import strip_accents_unicode

def clean_twitter(s):
    """ Cleans Twitter specific issues 
    
    Can you think of what else you might need to add here?
    """
    s = sub(r'@\w+', '', s) #remove @ mentions from tweets    
    return s

def preprocessor(s):
    """ For all basic string cleanup. 
    
    Think of what you can add to this to improve things. What is
    specific to your goal, how can you transform the text. Add tokens,
    remove things, unify things. 
    """
    s = clean_html(s)
    s = strip_accents_unicode(s.lower())
    s = clean_twitter(s)
    return s

In [ ]:
import re
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS

def analyzer(s, ngram_range = (1,2)):
    """ Does everything to turn raw documents into tokens.  
    
    Note: None of the above tokenizers are implemented!
    """
    s = preprocessor(s)
    pattern = re.compile(r"(?u)\b\w\w+\b")
    unigrams = pattern.findall(s)
    unigrams = [u for u in unigrams if u not in ENGLISH_STOP_WORDS]
    tokens = ngrammer(unigrams, ngram_range)
    return tokens

In [ ]:
X = pd.read_csv('kaggle/train.csv').tweet
y = pd.read_csv('kaggle/train.csv').label

cutoff = 1750
X_train, X_test, y_train, y_test = X[0:cutoff], X[cutoff:], y[0:cutoff], y[cutoff:]

In [118]:
import langdetect
from langdetect.lang_detect_exception import LangDetectException

def detect(st):
    try: 
        return langdetect.detect(st)
    except LangDetectException:
        return None

In [121]:
df = pd.read_csv('kaggle/train.csv')
langs = df.tweet.map(detect)
df[langs == 'en'].to_csv('kaggle/train-en.csv', quoting=csv.QUOTE_ALL, index=False)

In [ ]:
# Turn the text data into a fixed-length vectors that can be used to train a linear model.

In [ ]:
from sklearn.linear_model import LogisticRegression

# SKLearn's Logistic Regression allows us to use an L1 
# penalty, turning it into a Lasso classifier.

# What does the C parameter do, according to the documentation? 
lasso = LogisticRegression(penalty='l1', C=1.)

In [ ]:
# Make your predictions for the test set (X_test/y_test)

preds = 

In [ ]:
# Take a look at the mistakes

false_pos, false_neg = get_errors(X_test, y_test, preds)